In [1]:
from logai.preprocess.partition import PartitionerConfig
from logai.applications.auto_log_summarization import AutoLogSummarization
from logai.algorithms.parsing_algo.drain import DrainParams
from logai.information_extraction.log_parser import LogParserConfig
from logai.preprocess.preprocess import PreprocessorConfig
from logai.dataloader.data_loader import DataLoaderConfig
from logai.applications.application_interfaces import WorkFlowConfig


TEST_LOG_PATH = "/Users/qcheng/workspace/gitsoma/logai/tests/logai/test_data/HealthApp_format_2000.csv"

config = WorkFlowConfig(
            data_loader_config=DataLoaderConfig(
                filepath=TEST_LOG_PATH,
                log_type='csv',
                dimensions={
                    "attributes" : ["Action", "ID"],
                    "body": ["Details"]
                },
                reader_args={
                    "on_bad_lines": "skip"
                }
            ),
            preprocessor_config=PreprocessorConfig(
                custom_delimiters_regex=None
            ),
            log_parser_config=LogParserConfig(
                parsing_algorithm='drain',
                parsing_algo_params=DrainParams(
                    sim_th=0.4
                )
            ),
        )

parsing_app = AutoLogSummarization(config)
parsing_app.execute()

In [2]:
# Check all parsing results
parsing_app.parsing_results.head(5)


,logline,parsed_logline,parameter_list,Action,ID
0,onStandStepChanged 3579,onStandStepChanged *,[3579],Step_LSC,30002312
1,onExtend:1514038530000 14 0 4,* * 0 4,"[onExtend:1514038530000, 14]",Step_LSC,30002312
2,onReceive action: android.intent.action.SCREEN_ON,onReceive action: *,[android.intent.action.SCREEN_ON],Step_StandReportReceiver,30002312
3,processHandleBroadcastAction action:android.in...,processHandleBroadcastAction *,[action:android.intent.action.SCREEN_ON],Step_LSC,30002312
4,flush sensor data,flush sensor data,[],Step_StandStepCounter,30002312


In [3]:
# Check log pattern for a given logline
logline = parsing_app.parsing_results['logline'].tolist()[0]
print("Target logline: " + logline)

log_pattern, dynamic_parameters = parsing_app.find_log_pattern(logline)

print("-------------------------------------------")
print("Log pattern is: " + log_pattern)
print("-------------------------------------------")
print("Dynamic parameters")

print(dynamic_parameters)

Target logline: onStandStepChanged 3579
-------------------------------------------
Log pattern is: onStandStepChanged *
-------------------------------------------
Dynamic parameters
     position  value_counts                                             values
0  POSITION_0           260  [3579, 3579, 3579, 3580, 3581, 3583, 3584, 358...


In [4]:
# Select target attributes, here us action and id as example.
action = "Step_LSC"
id = 30002312

res = parsing_app.parsing_results

patterns = res.loc[(res['Action'] == action) & (res["ID"] == id)]['parsed_logline']

for p in patterns:
    dynamic_parameters = parsing_app.get_parameter_list(p)

    print("-------------------------------------------")
    print("Log pattern is: " + p)
    print("-------------------------------------------")
    print("Dynamic parameters")
    print(dynamic_parameters)
